In [1]:
model_path ="mistralai/Mistral-7B-Instruct-v0.2"

In [2]:
model_kwargs = {
        "load_in_4bit" : True           
}

In [3]:
from transformers import AutoModel, AutoConfig, BitsAndBytesConfig

In [4]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [6]:
model = AutoModel.from_pretrained(model_path, load_in_4bit=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
for nm in model.named_modules(): print(nm[0])

In [8]:
print_trainable_parameters(model)

trainable params: 131338240 || all params: 3620999168 || trainable%: 3.63


In [9]:
from peft import LoraConfig, get_peft_model

In [10]:
lora_config = LoraConfig(
    r = 16, 
    lora_alpha= 16, 
    target_modules= ["q_proj", "v_proj"],
    lora_dropout=.1, 
    bias="none"
)
    

In [17]:
lora_model = get_peft_model(model, lora_config)

In [12]:
from transformers import TrainingArguments, Trainer


In [13]:
batch_size = 1

In [19]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "perfModel", 
    evaluation_strategy="epoch", 
    save_strategy="epoch", 
    learning_rate=5e-3, 
    num_train_epochs=5, 
    per_device_eval_batch_size=batch_size, 
    gradient_accumulation_steps=2, 
    fp16=True,
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy"
)


In [20]:
from datasets import load_dataset                   


dataset = load_dataset('json', data_files="./hfdataset/hf_datafile.json")

In [21]:
trainer = Trainer(
    lora_model, 
    args, 
    train_dataset=dataset)

trainer.train()
    

MlflowException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Content-Length': '224', 'Content-Type': 'application/json', 'Date': 'Wed, 24 Apr 2024 01:46:14 GMT'})
HTTP response body: {"error":"Failed to update/create params in db: pq: value too long for type character varying(250)","code":13,"message":"Failed to update/create params in db: pq: value too long for type character varying(250)","details":[]}
